# 3.1 Imports

In [43]:
# import libraries
import numpy as np
import pandas as pd

import urllib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import time
from PIL import Image

from sklearn.metrics.pairwise import pairwise_distances

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [44]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [45]:
# create function
# open web browser

def configure_driver():
    # incognito window
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    
    # add the argument and make the browser Headless.
    # Chrome does not work well with headless
    # chrome_options.add_argument("--headless")

    # open web browser
    driver = webdriver.Chrome('/Users/abrahamleung/Documents/chromedriver', options=chrome_options)
    return driver

In [46]:
# list of categories
categories = [
    'activewear',
    'jackets',
    'sweatshirts-hoodies'
]

In [24]:
# load data
df = pd.read_csv('data/activewear.csv')

# 3.2 Amazon Search Engine Setup

In [11]:
# open web browser
driver = configure_driver()

In [12]:
# search engine
search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

In [13]:
# input website
driver.get(search_engine)

# may need time sleep
time.sleep(0.5)

In [18]:
# Choose HKD

# select option
option = driver.find_element_by_id('icp-touch-link-cop')
option.click()

# may need time sleep
time.sleep(0.5)

# select currency setting
currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
currency.click()

# select hkd as currency
hkd = driver.find_element_by_xpath('//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]')
hkd.click()

# save change
save = driver.find_element_by_id('icp-btn-save')
save.click()

# may need time sleep
time.sleep(0.5)

In [19]:
# select 'women's fashion' as department

department = driver.find_element_by_xpath('//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]')
department.click()

# may need time sleep
time.sleep(0.1)

In [6]:
# (unsuccessful) create function
# driver cannot serve as a variable
# it means the driver cannot be input nor output through a function

# Amazon search engine setup

def amazon_setup():

    # open web browser
    driver = configure_driver()

    # search engine
    search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

    # input website
    driver.get(search_engine)

    # may need time sleep
    time.sleep(0.5)

    # Choose HKD

    # select option
    option = driver.find_element_by_id('icp-touch-link-cop')
    option.click()

    # may need time sleep
    time.sleep(0.5)

    # select currency setting
    currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
    currency.click()

    # select hkd as currency
    hkd = driver.find_element_by_xpath(
        '//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]'
    )
    hkd.click()

    # save change
    save = driver.find_element_by_id('icp-btn-save')
    save.click()

    # may need time sleep
    time.sleep(0.5)

    # select 'women's fashion' as department
    department = driver.find_element_by_xpath(
        '//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]'
    )
    department.click()

    # may need time sleep
    time.sleep(0.1)
    
    return driver

# 3.3 Scraping of Every Link

In [21]:
# input search item

search_bar = driver.find_element_by_id('twotabsearchtextbox')
search_bar.send_keys(key_words)
search_bar.send_keys(Keys.ENTER)

# may need time sleep
time.sleep(0.5)

In [22]:
# get links

links = []
link_paths = driver.find_elements_by_xpath('//div/h2/a[@class="a-link-normal a-text-normal"]')
for link in link_paths:
    x = link.get_attribute('href')
    links.append(x)

In [11]:
# create function
# scraping every link

def all_links(driver, key_words):

    # input search item
    search_bar = driver.find_element_by_id('twotabsearchtextbox')
    search_bar.send_keys(key_words)
    search_bar.send_keys(Keys.ENTER)

    # may need time sleep
    time.sleep(0.5)

    # get links
    links = []
    link_paths = driver.find_elements_by_xpath('//div/h2/a[@class="a-link-normal a-text-normal"]')
    for link in link_paths:
        x = link.get_attribute('href')
        links.append(x)
    
    return links

# 3.4 Scraping of Search Result Details

In [27]:
# testing
# url

URL = links[0]
URL

'https://www.amazon.com/CUPSHE-Womens-Floral-Halter-Swimsuit/dp/B07S223TYL/ref=sr_1_1?dchild=1&keywords=black+cutout+print+bikini&qid=1624866037&s=fashion-womens-intl-ship&sr=1-1'

In [28]:
# input website
driver.get(URL)

In [29]:
# product names
try:
    name = driver.find_elements_by_tag_name('h1')
    name = name[0].text
except:
    name = None

In [30]:
name

"CUPSHE Women's One Piece Swimsuit Cutout Halter Lace Up Twist Bathing Suit"

In [31]:
# product price
try:
    price = driver.find_elements_by_xpath('//td[@class="a-span12"]')
    price = price[0].text
except:
    price = None

In [32]:
price

'HKD 237.32'

In [39]:
# testing
index = 0

In [40]:
# product image
try:
    # size
    img_width,img_height = 300,300

    # get image
    img = driver.find_elements_by_xpath('//img[@id="landingImage"]')
    img = img[0]

    # 'src' = get image source
    src = img.get_attribute('src')

    # download image
    urllib.request.urlretrieve(src, f'image/Amazon/{index}.png')

    # resize image (smaller size)
    ori_img = Image.open(f'image/Amazon/{index}.png')
    resize_img = ori_img.resize((img_width,img_height))
    resize_img.save(f'image/Amazon/{index}.png')

    img_file = f'image/Amazon/{index}.png'
except:
    img_file = None


In [8]:
# create function
# search result details

def scrape_details(index, link):

    # input website
    driver.get(link)

    # product names
    try:
        name = driver.find_elements_by_tag_name('h1')
        name = name[0].text
    except:
        name = None

    # product price
    try:
        price = driver.find_elements_by_xpath('//td[@class="a-span12"]')
        price = price[0].text
    except:
        price = None
    
    # product image
    try:
        # size
        img_width,img_height = 300,300

        # get image
        img = driver.find_elements_by_xpath('//img[@id="landingImage"]')
        img = img[0]

        # 'src' = get image source
        src = img.get_attribute('src')

        # download image
        urllib.request.urlretrieve(src, f'image/Amazon/{index}.png')

        # resize image (smaller size)
        ori_img = Image.open(f'image/Amazon/{index}.png')
        resize_img = ori_img.resize((img_width,img_height))
        resize_img.save(f'image/Amazon/{index}.png')

        img_file = f'image/Amazon/{index}.png'
    except:
        img_file = None

    return name, price, img_file

In [9]:
# scrape every link

names = []
prices = []
img_files = []
index = 0

for link in links:
    
    # scrape name, price and img_file
    name, price, img_file = scrape_details(index, link)

    # append to lists
    names.append(name)
    prices.append(price)
    img_files.append(img_file)

    # update index
    index += 1

# create dataframe

df = pd.DataFrame({
    'name': names,
    'price': prices,
    'img_file': img_files,
    'url': links
})

NameError: name 'links' is not defined

# 3.5 Product Recommendations

In [47]:
# load model
# as no model training required, set 'compile=False'

model = load_model('cnn_model.h5', compile=False)

In [48]:
# create function
# feature extraction for image

def get_embedding(model, img_file):

    # reshape
    img = image.load_img(img_file, target_size=(224, 224))

    # img to numpy array
    x = image.img_to_array(img)

    # expand dim (1, w, h)
    x = np.expand_dims(x, axis=0)

    # Preprocess input
    x = preprocess_input(x)

    return model.predict(x).reshape(-1)

In [49]:
# Function that get product recommendations based on the cosine similarity score of product image

def get_recommender(idx, df, similarity, top_n):

    # create dataframe for indices
    indices = pd.Series(range(len(df)), index=df.index)

    # select the target image (default is index 0)
    target_index = indices[idx]

    # calculate the similarity scores
    similarity_scores = list(enumerate(similarity[target_index]))

    # sort the result 
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # select the top n items
    similarity_scores = similarity_scores[1:top_n+1]

    # select the indices of top n items
    top_indices = [i[0] for i in similarity_scores]

    # idx_sim = [i[1] for i in sim_scores]

    return indices.iloc[top_indices].index

In [58]:
# make a dataframe duplicate
df_copy = df

# get image embeddings for each image
map_embeddings = df_copy['img_file'].apply(lambda img: get_embedding(model, img))

# convert series of lists to dataframe
df_embeddings = map_embeddings.apply(pd.Series)

In [59]:
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,16.656422,1.759933,0.396486,5.663714,0.000000,1.727282,0.442207,0.000000,0.000000,...,0.000000,0.000000,2.258237,2.662622,3.287225,0.0,12.032290,3.004779,0.000000,1.093109
1,0.000000,7.160003,1.574659,4.338456,2.551096,2.673633,4.204260,6.550857,0.942263,2.079585,...,0.975850,0.997066,2.406110,25.856796,3.466363,0.0,7.697368,0.309879,2.489410,7.677667
2,1.831643,22.057829,2.530438,1.242392,0.000000,0.000000,1.320725,3.649347,0.030288,0.000000,...,7.023321,2.911746,0.000000,1.198196,2.623420,0.0,2.178410,1.167908,3.153160,12.983468
3,3.890997,2.239640,0.561372,6.952099,2.865389,3.875372,6.084309,2.111632,2.495424,4.386785,...,0.406559,2.324595,1.143666,17.603485,2.035505,0.0,1.155617,2.491142,0.000000,5.401934
4,0.479622,3.835210,3.169219,5.755448,1.441075,5.746672,3.464339,4.615533,4.365491,2.138535,...,12.700304,0.725670,2.027468,11.041115,0.000000,0.0,0.997130,1.629200,1.458257,10.393693


In [60]:
## compute similarity between items
# calculate distance matrix
similarity = 1 - pairwise_distances(df_embeddings, metric='cosine')

In [64]:
similarity[0]

array([1.        , 0.5165263 , 0.5369592 , 0.51267225, 0.5224726 ,
       0.5031241 , 0.5237402 , 0.51745456, 0.5408669 , 0.47479218,
       0.48267484, 0.46357816, 0.50001   , 0.5165371 , 0.5172481 ,
       0.58180815, 0.49073946, 0.5257547 , 0.5269011 , 0.5430988 ,
       0.55302197, 0.57794875, 0.5942491 , 0.49452055, 0.5181655 ,
       0.5339118 , 0.552034  , 0.5226602 , 0.5101911 , 0.49412894,
       0.52358085, 0.5315855 , 0.51819026, 0.4978131 , 0.5014058 ,
       0.47400868, 0.5076667 , 0.5177233 , 0.494326  , 0.48988938,
       0.50520056, 0.5170815 , 0.49531442, 0.5356687 , 0.52722985,
       0.5314058 , 0.5180114 , 0.50884783, 0.4995228 ], dtype=float32)

In [65]:
# create dataframe for indices
indices = pd.Series(range(len(df)), index=df.index)

In [69]:
# compute recommendations' indices
recommend_indices = get_recommender(0, df, 2)

# create lists
recommend_names = []
recommend_prices = []
recommend_img_files = []
recommend_urls = []

for i in recommend_indices:
    # recommend_names.append(df.iloc[i]['name'])
    # recommend_prices.append(df.iloc[i]['price'])
    # recommend_urls.append(df.iloc[i]['url'])

    # save the img_files separately
    # use 'i-1' because the target item was appended at the front of dataframe
    # the corresponding image index has to minus 1
    img = Image.open(f'image/Amazon/{i-1}.png')

    img_name = df.iloc[0]['img_file'][6:-4] + str(i) + '.png'
    img.save(f'image/recommendations/{img_name}.png')

    img_file = f'image/recommendations/{img_name}.png'

    # append img_file's new path
    recommend_img_files.append(img_file)

In [50]:
# create products recommender

def product_recommender(i,ck_target,df_amazon,model):
    df_target = pd.DataFrame({
        'name': [ck_target['name']],
        'price': [ck_target['price']],
        'img_file': [ck_target['img_file']],
        'url': [ck_target['url']]
    })

    # append amazon dataframe to the target dataframe
    df = df_target.append(df_amazon, ignore_index=True)

    # make a dataframe duplicate
    df_copy = df

    # get image embeddings for each image
    map_embeddings = df_copy['img_file'].apply(lambda img: get_embedding(model, img))

    # convert series of lists to dataframe
    df_embeddings = map_embeddings.apply(pd.Series)

    ## compute similarity between items
    # calculate distance matrix
    similarity = 1 - pairwise_distances(df_embeddings, metric='cosine')

    # create dataframe for indices
    indices = pd.Series(range(len(df)), index=df.index)

    # compute recommendations' indices
    recommend_indices = get_recommender(0, df, similarity, 2)

    # create lists
    recommend_names = []
    recommend_prices = []
    recommend_img_files = []
    recommend_urls = []

    for j in recommend_indices:
        recommend_names.append(df.iloc[j]['name'])
        recommend_prices.append(df.iloc[j]['price'])
        recommend_urls.append(df.iloc[j]['url'])

        # save the img_files separately
        # use 'i-1' because the target item was appended at the front of dataframe
        # the corresponding image index has to minus 1
        img = Image.open(f'image/Amazon/{j-1}.png')

        img_name = ck_target['name'] + str(j)
        img.save(f'image/recommendations/{img_name}.png')

        img_file = f'image/recommendations/{img_name}.png'

        # append img_file's new path
        recommend_img_files.append(img_file)
    return recommend_names, recommend_prices, recommend_img_files, recommend_urls

# 3.6 Apply to All Categories

# 3.6.1 Activewear

In [9]:
## Amazon search engine setup

# open web browser
driver = configure_driver()

# search engine
search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

# input website
driver.get(search_engine)

# may need time sleep
time.sleep(0.5)

# Choose HKD

# select option
option = driver.find_element_by_id('icp-touch-link-cop')
option.click()

# may need time sleep
time.sleep(0.5)

# select currency setting
currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
currency.click()

# select hkd as currency
hkd = driver.find_element_by_xpath(
    '//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]'
)
hkd.click()

# save change
save = driver.find_element_by_id('icp-btn-save')
save.click()

# may need time sleep
time.sleep(0.5)

# select 'women's fashion' as department
department = driver.find_element_by_xpath(
    '//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]'
)
department.click()

# may need time sleep
time.sleep(0.1)

In [10]:
# activewear
category = 'activewear'
    
# load data
df_ck = pd.read_csv(f'data/{category}.csv')     

list_recommend_names = []
list_recommend_prices = []
list_recommend_img_files = []
list_recommend_urls = []

## scraping all links
for i,key_words in enumerate(df_ck['key_words']):

    start_time = time.time()

    # input website
    URL = f'https://www.amazon.com/s?k={key_words}&i=fashion-womens-intl-ship&ref=nb_sb_noss_2'
    driver.get(URL)

    # may need time sleep
    time.sleep(0.5)

    # get links
    links = []
    link_paths = driver.find_elements_by_xpath('//div/h2/a[@class="a-link-normal a-text-normal"]')
    for link in link_paths:
        x = link.get_attribute('href')
        links.append(x)
    
    ## scrape every product details

    names = []
    prices = []
    img_files = []
    index = 0

    for link in links:
        
        # input website
        driver.get(link)

        # product names
        try:
            name = driver.find_elements_by_tag_name('h1')
            name = name[0].text
        except:
            name = None

        # product price
        try:
            price = driver.find_elements_by_xpath('//td[@class="a-span12"]')
            price = price[0].text
        except:
            price = None
        
        # product image
        try:
            # size
            img_width,img_height = 300,300

            # get image
            img = driver.find_elements_by_xpath('//img[@id="landingImage"]')
            img = img[0]

            # 'src' = get image source
            src = img.get_attribute('src')

            # download image
            urllib.request.urlretrieve(src, f'image/Amazon/{index}.png')

            # resize image (smaller size)
            ori_img = Image.open(f'image/Amazon/{index}.png')
            resize_img = ori_img.resize((img_width,img_height))
            resize_img.save(f'image/Amazon/{index}.png')

            img_file = f'image/Amazon/{index}.png'
        except:
            img_file = None

        # append to lists
        names.append(name)
        prices.append(price)
        img_files.append(img_file)

        # update index
        index += 1

    # create dataframe
    df_amazon = pd.DataFrame({
        'name': names,
        'price': prices,
        'img_file': img_files,
        'url': links
    })

    # remove row with missing values
    df_amazon.dropna(inplace=True)
    df_amazon.reset_index(drop=True, inplace=True)

    # products recommender
    recommend_names, recommend_prices, recommend_img_files, recommend_urls = product_recommender(
        i,df_ck.iloc[i],df_amazon,model
    )

    ## output lists for creating dataframe for final deployment use
    list_recommend_names.append(recommend_names)
    list_recommend_prices.append(recommend_prices)
    list_recommend_img_files.append(recommend_img_files)
    list_recommend_urls.append(recommend_urls)
    
    end_time = time.time()
    print(f'({i+1}/{len(df_ck)}) finished, execution time: {round(end_time - start_time,2)}s')

print('Execution Completed')

(1/12) finished, execution time: 118.63s
(2/12) finished, execution time: 134.76s
(3/12) finished, execution time: 130.51s
(4/12) finished, execution time: 81.45s
(5/12) finished, execution time: 125.91s
(6/12) finished, execution time: 136.77s
(7/12) finished, execution time: 124.14s
(8/12) finished, execution time: 136.37s
(9/12) finished, execution time: 128.9s
(10/12) finished, execution time: 130.68s
(11/12) finished, execution time: 39.93s
(12/12) finished, execution time: 138.3s
Execution Completed


In [11]:
# create dataframe

names = df_ck['name'].to_list()
prices = df_ck['price'].to_list()
img_files = df_ck['img_file'].to_list()
urls = df_ck['url'].to_list()

df_recommender = pd.DataFrame({
    'name': names,
    'price': prices,
    'img_file': img_files,
    'url': urls,
    'recommend_names': list_recommend_names,
    'recommend_prices': list_recommend_prices,
    'recommend_img_files': list_recommend_img_files,
    'recommend_urls': list_recommend_urls
})

# save as csv
df_recommender.to_csv(f'data/{category}_recommender.csv', index=False)

In [36]:
# activewear
category = 'activewear'

# load data
df_recommender = pd.read_csv(f'data/{category}_recommender.csv')
df_recommender.head()

,name,price,img_file,url,recommend_name1,recommend_name2,recommend_price1,recommend_price2,recommend_img_file1,recommend_img_file2,recommend_url1,recommend_url2
0,CK ONE TANGA BIKINI,HKD 392.00,image/CK ONE TANGA BIKINI.png,https://www.calvinklein.com/hk/en/ck-one-tanga...,SheIn Women's 2 Pieces Leopard Print Strap Cut...,OMKAGI Women's 2 Pieces Bandeau Bikini Swimsui...,HKD 149.86 - HKD 165.65,HKD 118.30 - HKD 244.56,image/recommendations/CK ONE TANGA BIKINI26.png,image/recommendations/CK ONE TANGA BIKINI9.png,https://www.amazon.com/SheIn-Womens-Pieces-Leo...,https://www.amazon.com/OMKAGI-Bandeau-Swimsuit...
1,CK ONE BRALETTE,HKD 392.00,image/CK ONE BRALETTE.png,https://www.calvinklein.com/hk/en/ck-one-brale...,SheIn Women's Sexy Straps Sleeveless Cutout Cr...,Amazon Brand - Mae Women's High-Neck Bralette ...,HKD 157.75,HKD 126.27,image/recommendations/CK ONE BRALETTE40.png,image/recommendations/CK ONE BRALETTE28.png,https://www.amazon.com/SheIn-Womens-Straps-Sle...,https://www.amazon.com/Amazon-Brand-Hi-Neck-Br...
2,ACTIVE ICON WOVEN SHORTS,HKD 413.00,image/ACTIVE ICON WOVEN SHORTS.png,https://www.calvinklein.com/hk/en/active-icon-...,Conceited Premium Ultra Soft Harem Shorts - Po...,Vibrant Women's Juniors Denim High Waist Cutof...,HKD 117.98 - HKD 149.55,HKD 157.75 - HKD 315.59,image/recommendations/ACTIVE ICON WOVEN SHORTS...,image/recommendations/ACTIVE ICON WOVEN SHORTS...,https://www.amazon.com/Premium-Ultra-Harem-Sho...,https://www.amazon.com/Vibrant-Womens-Juniors-...
3,ACTIVE ICON FULL LENGTH ALL-OVER PRINT LEGGINGS,HKD 483.00,image/ACTIVE ICON FULL LENGTH ALL-OVER PRINT L...,https://www.calvinklein.com/hk/en/active-icon-...,Calvin Klein Women's Premium Performance High ...,Hard Tail Women's High Rise Ankle Performance ...,HKD 231.70 - HKD 469.55,HKD 678.68,image/recommendations/ACTIVE ICON FULL LENGTH ...,image/recommendations/ACTIVE ICON FULL LENGTH ...,https://www.amazon.com/Calvin-Klein-Performanc...,https://www.amazon.com/Hard-Tail-Womens-Perfor...
4,LACE LIGHTWEAR JACKET,HKD 903.00,image/LACE LIGHTWEAR JACKET.png,https://www.calvinklein.com/hk/en/lace-lightwe...,Hollywood Star Fashion Womens Basic Button Dow...,Riders by Lee Indigo Women's Plus Size Easy Ca...,HKD 125.48 - HKD 275.81,HKD 144.10 - HKD 183.24,image/recommendations/LACE LIGHTWEAR JACKET4.png,image/recommendations/LACE LIGHTWEAR JACKET28.png,https://www.amazon.com/Hollywood-Star-Fashion-...,https://www.amazon.com/Riders-Lee-Indigo-Women...


In [41]:
print(df_recommender['recommend_name1'].iloc[11])
print(df_recommender['recommend_name2'].iloc[11])

Calvin Klein Women's Premium Performance High Waist Moisture Wicking Legging
90 Degree By Reflex High Waist Squat Proof Interlink Leggings for Women


In [40]:
def name2(f):
    if pd.isna(f['recommend_name2']):
        if '\', "' in f['recommend_name1']:
            return f['recommend_name1'].strip('"').split('\', "')[1]
        if "\", '" in f['recommend_name1']:
            return f['recommend_name1'].strip("'").split("\", '")[1]
    else:
        return f['recommend_name2']

df_recommender['recommend_name2'] = df_recommender.apply(name2, axis=1)

def name1(x):
    if '\', "' in x:
        return x.split('\', "')[0]
    if "\", '" in x:
        return x.split("\", '")[0]
    else:
        return x

df_recommender['recommend_name1'] = df_recommender['recommend_name1'].apply(lambda x: name1(x))

df_recommender

,name,price,img_file,url,recommend_name1,recommend_name2,recommend_price1,recommend_price2,recommend_img_file1,recommend_img_file2,recommend_url1,recommend_url2
0,CK ONE TANGA BIKINI,HKD 392.00,image/CK ONE TANGA BIKINI.png,https://www.calvinklein.com/hk/en/ck-one-tanga...,SheIn Women's 2 Pieces Leopard Print Strap Cut...,OMKAGI Women's 2 Pieces Bandeau Bikini Swimsui...,HKD 149.86 - HKD 165.65,HKD 118.30 - HKD 244.56,image/recommendations/CK ONE TANGA BIKINI26.png,image/recommendations/CK ONE TANGA BIKINI9.png,https://www.amazon.com/SheIn-Womens-Pieces-Leo...,https://www.amazon.com/OMKAGI-Bandeau-Swimsuit...
1,CK ONE BRALETTE,HKD 392.00,image/CK ONE BRALETTE.png,https://www.calvinklein.com/hk/en/ck-one-brale...,SheIn Women's Sexy Straps Sleeveless Cutout Cr...,Amazon Brand - Mae Women's High-Neck Bralette ...,HKD 157.75,HKD 126.27,image/recommendations/CK ONE BRALETTE40.png,image/recommendations/CK ONE BRALETTE28.png,https://www.amazon.com/SheIn-Womens-Straps-Sle...,https://www.amazon.com/Amazon-Brand-Hi-Neck-Br...
2,ACTIVE ICON WOVEN SHORTS,HKD 413.00,image/ACTIVE ICON WOVEN SHORTS.png,https://www.calvinklein.com/hk/en/active-icon-...,Conceited Premium Ultra Soft Harem Shorts - Po...,Vibrant Women's Juniors Denim High Waist Cutof...,HKD 117.98 - HKD 149.55,HKD 157.75 - HKD 315.59,image/recommendations/ACTIVE ICON WOVEN SHORTS...,image/recommendations/ACTIVE ICON WOVEN SHORTS...,https://www.amazon.com/Premium-Ultra-Harem-Sho...,https://www.amazon.com/Vibrant-Womens-Juniors-...
3,ACTIVE ICON FULL LENGTH ALL-OVER PRINT LEGGINGS,HKD 483.00,image/ACTIVE ICON FULL LENGTH ALL-OVER PRINT L...,https://www.calvinklein.com/hk/en/active-icon-...,Calvin Klein Women's Premium Performance High ...,Hard Tail Women's High Rise Ankle Performance ...,HKD 231.70 - HKD 469.55,HKD 678.68,image/recommendations/ACTIVE ICON FULL LENGTH ...,image/recommendations/ACTIVE ICON FULL LENGTH ...,https://www.amazon.com/Calvin-Klein-Performanc...,https://www.amazon.com/Hard-Tail-Womens-Perfor...
4,LACE LIGHTWEAR JACKET,HKD 903.00,image/LACE LIGHTWEAR JACKET.png,https://www.calvinklein.com/hk/en/lace-lightwe...,Hollywood Star Fashion Womens Basic Button Dow...,Riders by Lee Indigo Women's Plus Size Easy Ca...,HKD 125.48 - HKD 275.81,HKD 144.10 - HKD 183.24,image/recommendations/LACE LIGHTWEAR JACKET4.png,image/recommendations/LACE LIGHTWEAR JACKET28.png,https://www.amazon.com/Hollywood-Star-Fashion-...,https://www.amazon.com/Riders-Lee-Indigo-Women...
5,CHINESE NEW YEAR CAPSULE PADDED JACKET WITH HOOD,"HKD 1,603.00",image/CHINESE NEW YEAR CAPSULE PADDED JACKET W...,https://www.calvinklein.com/hk/en/chinese-new-...,Columbia Womens Hexbreaker Long Down Jacket,Calvin Klein Women's Mid-Weight Diamond Quilte...,"HKD 1,177.90 - HKD 2,288.57","HKD 599.37 - HKD 1,104.75",image/recommendations/CHINESE NEW YEAR CAPSULE...,image/recommendations/CHINESE NEW YEAR CAPSULE...,https://www.amazon.com/Columbia-Womens-Hexbrea...,https://www.amazon.com/Calvin-Klein-Classic-Di...
6,OMBRE LOGO SHORT SLEEVE TEE,HKD 343.00,image/OMBRE LOGO SHORT SLEEVE TEE.png,https://www.calvinklein.com/hk/en/ombre-logo-s...,Nautica Women's Easy Comfort Supersoft 100% Co...,Nautica Women's Boatneck 3/4 Sleeve 100% Cotto...,HKD 43.80 - HKD 157.75,HKD 236.67,image/recommendations/OMBRE LOGO SHORT SLEEVE ...,image/recommendations/OMBRE LOGO SHORT SLEEVE ...,https://www.amazon.com/Nautica-Comfort-Superso...,https://www.amazon.com/Nautica-Womens-Solids-S...
7,CHINESE NEW YEAR CAPSULE PADDED JACKET WITH HOOD,"HKD 1,603.00",image/CHINESE NEW YEAR CAPSULE PADDED JACKET W...,https://www.calvinklein.com/hk/en/chinese-new-...,Bulotus Women's Long Sleeve Cowl Neck Casual T...,GK-O Anime Danganronpa Monokuma Warm Thicken H...,HKD 205.10 - HKD 236.67,HKD 220.89,image/recommendations/CHINESE NEW YEAR CAPSULE...,image/recommendations/CHINESE NEW YEAR CAPSULE...,https://www.amazon.com/Bulotus-Womens-Sleeve-P...,https://www.amazon.com/GK-Danganronpa-Monokuma...
8,REFLECTIVE COLOR BLOCK WINDBREAKER,HKD 763.00,image/R

In [12]:
# have to check the index 2, 4, 7, 11 carefully

df_recommender

,name,price,img_file,url,recommend_name1,recommend_name2,recommend_price1,recommend_price2,recommend_img_file1,recommend_img_file2,recommend_url1,recommend_url2
0,CK ONE TANGA BIKINI,HKD 392.00,image/CK ONE TANGA BIKINI.png,https://www.calvinklein.com/hk/en/ck-one-tanga...,SheIn Women's 2 Pieces Leopard Print Strap Cut...,SheIn Women's 2 Pieces Leopard Print Strap Cut...,HKD 149.86 - HKD 165.65,HKD 118.30 - HKD 244.56,image/recommendations/CK ONE TANGA BIKINI26.png,image/recommendations/CK ONE TANGA BIKINI9.png,https://www.amazon.com/SheIn-Womens-Pieces-Leo...,https://www.amazon.com/OMKAGI-Bandeau-Swimsuit...
1,CK ONE BRALETTE,HKD 392.00,image/CK ONE BRALETTE.png,https://www.calvinklein.com/hk/en/ck-one-brale...,SheIn Women's Sexy Straps Sleeveless Cutout Cr...,SheIn Women's Sexy Straps Sleeveless Cutout Cr...,HKD 157.75,HKD 126.27,image/recommendations/CK ONE BRALETTE40.png,image/recommendations/CK ONE BRALETTE28.png,https://www.amazon.com/SheIn-Womens-Straps-Sle...,https://www.amazon.com/Amazon-Brand-Hi-Neck-Br...
2,ACTIVE ICON WOVEN SHORTS,HKD 413.00,image/ACTIVE ICON WOVEN SHORTS.png,https://www.calvinklein.com/hk/en/active-icon-...,Conceited Premium Ultra Soft Harem Shorts - Po...,Conceited Premium Ultra Soft Harem Shorts - Po...,HKD 117.98 - HKD 149.55,HKD 157.75 - HKD 315.59,image/recommendations/ACTIVE ICON WOVEN SHORTS...,image/recommendations/ACTIVE ICON WOVEN SHORTS...,https://www.amazon.com/Premium-Ultra-Harem-Sho...,https://www.amazon.com/Vibrant-Womens-Juniors-...
3,ACTIVE ICON FULL LENGTH ALL-OVER PRINT LEGGINGS,HKD 483.00,image/ACTIVE ICON FULL LENGTH ALL-OVER PRINT L...,https://www.calvinklein.com/hk/en/active-icon-...,Calvin Klein Women's Premium Performance High ...,Calvin Klein Women's Premium Performance High ...,HKD 231.70 - HKD 469.55,HKD 678.68,image/recommendations/ACTIVE ICON FULL LENGTH ...,image/recommendations/ACTIVE ICON FULL LENGTH ...,https://www.amazon.com/Calvin-Klein-Performanc...,https://www.amazon.com/Hard-Tail-Womens-Perfor...
4,LACE LIGHTWEAR JACKET,HKD 903.00,image/LACE LIGHTWEAR JACKET.png,https://www.calvinklein.com/hk/en/lace-lightwe...,Hollywood Star Fashion Womens Basic Button Dow...,Hollywood Star Fashion Womens Basic Button Dow...,HKD 125.48 - HKD 275.81,HKD 144.10 - HKD 183.24,image/recommendations/LACE LIGHTWEAR JACKET4.png,image/recommendations/LACE LIGHTWEAR JACKET28.png,https://www.amazon.com/Hollywood-Star-Fashion-...,https://www.amazon.com/Riders-Lee-Indigo-Women...
5,CHINESE NEW YEAR CAPSULE PADDED JACKET WITH HOOD,"HKD 1,603.00",image/CHINESE NEW YEAR CAPSULE PADDED JACKET W...,https://www.calvinklein.com/hk/en/chinese-new-...,Columbia Womens Hexbreaker Long Down Jacket,Columbia Womens Hexbreaker Long Down Jacket,"HKD 1,177.90 - HKD 2,288.57","HKD 599.37 - HKD 1,104.75",image/recommendations/CHINESE NEW YEAR CAPSULE...,image/recommendations/CHINESE NEW YEAR CAPSULE...,https://www.amazon.com/Columbia-Womens-Hexbrea...,https://www.amazon.com/Calvin-Klein-Classic-Di...
6,OMBRE LOGO SHORT SLEEVE TEE,HKD 343.00,image/OMBRE LOGO SHORT SLEEVE TEE.png,https://www.calvinklein.com/hk/en/ombre-logo-s...,Nautica Women's Easy Comfort Supersoft 100% Co...,Nautica Women's Easy Comfort Supersoft 100% Co...,HKD 43.80 - HKD 157.75,HKD 236.67,image/recommendations/OMBRE LOGO SHORT SLEEVE ...,image/recommendations/OMBRE LOGO SHORT SLEEVE ...,https://www.amazon.com/Nautica-Comfort-Superso...,https://www.amazon.com/Nautica-Womens-Solids-S...
7,CHINESE NEW YEAR CAPSULE PADDED JACKET WITH HOOD,"HKD 1,603.00",image/CHINESE NEW YEAR CAPSULE PADDED JACKET W...,https://www.calvinklein.com/hk/en/chinese-new-...,Bulotus Women's Long Sleeve Cowl Neck Casual T...,Bulotus Women's Long Sleeve Cowl Neck Casual T...,HKD 205.10 - HKD 236.67,HKD 220.89,image/recommendations/CHINESE NEW YEAR CAPSULE...,image/recommendations/CHINESE NEW YEAR CAPSULE...,https://www.amazon.com/Bulotus-Womens-Sleeve-P...,https://www.amazon.com/GK-Danganronpa-Monokuma...
8,REFLECTIVE COLOR BLOCK WINDBREAKER,HKD 763.00,image/REFLECT

In [16]:
def remove_list(x):
    if '", "' in x:
        return x.strip('["').strip('"]').split('", "')
    if "', '" in x:
        return x.strip("['").strip("']").split("', '")
    if '\', "' in x:
        return x.strip("['").strip('"]').split('\', "')
    if "\", '" in x:
        return x.strip('["').strip("']").split("\", '")

In [18]:
# expanding recommendations into separate columns

recommend_columns = ['recommend_names', 'recommend_prices', 'recommend_img_files', 'recommend_urls']

for column in recommend_columns:
    df_recommender[f'{column[:-1]}1'] = df_recommender[column].apply(lambda x: remove_list(x)[0])
    df_recommender[f'{column[:-1]}2'] = df_recommender[column].apply(
        lambda x: remove_list(x)[1] if len(remove_list(x)) == 2 else None
    )
    df_recommender.drop([column], axis=1, inplace=True)

In [42]:
# save as csv
df_recommender.to_csv(f'data/{category}_recommender.csv', index=False)

# 3.6.2 Jackets

In [51]:
## Amazon search engine setup

# open web browser
driver = configure_driver()

# search engine
search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

# input website
driver.get(search_engine)

# may need time sleep
time.sleep(0.5)

# Choose HKD

# select option
option = driver.find_element_by_id('icp-touch-link-cop')
option.click()

# may need time sleep
time.sleep(0.5)

# select currency setting
currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
currency.click()

# select hkd as currency
hkd = driver.find_element_by_xpath(
    '//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]'
)
hkd.click()

# save change
save = driver.find_element_by_id('icp-btn-save')
save.click()

# may need time sleep
time.sleep(0.5)

# select 'women's fashion' as department
department = driver.find_element_by_xpath(
    '//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]'
)
department.click()

# may need time sleep
time.sleep(0.1)

# activewear
category = 'jackets'
    
# load data
df_ck = pd.read_csv(f'data/{category}.csv')     

list_recommend_names = []
list_recommend_prices = []
list_recommend_img_files = []
list_recommend_urls = []

## scraping all links
for i,key_words in enumerate(df_ck['key_words']):

    start_time = time.time()

    # input website
    URL = f'https://www.amazon.com/s?k={key_words}&i=fashion-womens-intl-ship&ref=nb_sb_noss_2'
    driver.get(URL)

    # may need time sleep
    time.sleep(0.5)

    # get links
    links = []
    link_paths = driver.find_elements_by_xpath('//div/h2/a[@class="a-link-normal a-text-normal"]')
    for link in link_paths:
        x = link.get_attribute('href')
        links.append(x)
    
    ## scrape every product details

    names = []
    prices = []
    img_files = []
    index = 0

    for link in links:
        
        # input website
        driver.get(link)

        # product names
        try:
            name = driver.find_elements_by_tag_name('h1')
            name = name[0].text
        except:
            name = None

        # product price
        try:
            price = driver.find_elements_by_xpath('//td[@class="a-span12"]')
            price = price[0].text
        except:
            price = None
        
        # product image
        try:
            # size
            img_width,img_height = 300,300

            # get image
            img = driver.find_elements_by_xpath('//img[@id="landingImage"]')
            img = img[0]

            # 'src' = get image source
            src = img.get_attribute('src')

            # download image
            urllib.request.urlretrieve(src, f'image/Amazon/{index}.png')

            # resize image (smaller size)
            ori_img = Image.open(f'image/Amazon/{index}.png')
            resize_img = ori_img.resize((img_width,img_height))
            resize_img.save(f'image/Amazon/{index}.png')

            img_file = f'image/Amazon/{index}.png'
        except:
            img_file = None

        # append to lists
        names.append(name)
        prices.append(price)
        img_files.append(img_file)

        # update index
        index += 1

    # create dataframe
    df_amazon = pd.DataFrame({
        'name': names,
        'price': prices,
        'img_file': img_files,
        'url': links
    })

    # remove row with missing values
    df_amazon.dropna(inplace=True)
    df_amazon.reset_index(drop=True, inplace=True)

    # products recommender
    recommend_names, recommend_prices, recommend_img_files, recommend_urls = product_recommender(
        i,df_ck.iloc[i],df_amazon,model
    )

    ## output lists for creating dataframe for final deployment use
    list_recommend_names.append(recommend_names)
    list_recommend_prices.append(recommend_prices)
    list_recommend_img_files.append(recommend_img_files)
    list_recommend_urls.append(recommend_urls)
    
    end_time = time.time()
    print(f'({i+1}/{len(df_ck)}) finished, execution time: {round(end_time - start_time,2)}s')

print('Execution Completed')

(1/11) finished, execution time: 114.13s
(2/11) finished, execution time: 40.65s
(3/11) finished, execution time: 168.77s
(4/11) finished, execution time: 134.43s
(5/11) finished, execution time: 102.58s
(6/11) finished, execution time: 152.66s
(7/11) finished, execution time: 128.65s
(8/11) finished, execution time: 146.5s
(9/11) finished, execution time: 140.95s
(10/11) finished, execution time: 108.46s
(11/11) finished, execution time: 172.84s
Execution Completed


In [58]:
# create dataframe

names = df_ck['name'].to_list()
prices = df_ck['price'].to_list()
img_files = df_ck['img_file'].to_list()
urls = df_ck['url'].to_list()

df_recommender = pd.DataFrame({
    'name': names,
    'price': prices,
    'img_file': img_files,
    'url': urls,
    'recommend_names': list_recommend_names,
    'recommend_prices': list_recommend_prices,
    'recommend_img_files': list_recommend_img_files,
    'recommend_urls': list_recommend_urls
})

expanding recommendations into separate columns

recommend_columns = ['recommend_names', 'recommend_prices', 'recommend_img_files', 'recommend_urls']

for column in recommend_columns:
    df_recommender[f'{column[:-1]}1'] = df_recommender[column].apply(lambda x: x[0])
    df_recommender[f'{column[:-1]}2'] = df_recommender[column].apply(
        lambda x: x[1] if len(x) == 2 else None
    )
    df_recommender.drop([column], axis=1, inplace=True)

# save as csv
df_recommender.to_csv(f'data/{category}_recommender.csv', index=False)

In [ ]:
# data cleaning of 'price'

# 3.6.3 Sweatshirts-Hoodies

In [ ]:
## Amazon search engine setup

# open web browser
driver = configure_driver()

# search engine
search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

# input website
driver.get(search_engine)

# may need time sleep
time.sleep(0.5)

# Choose HKD

# select option
option = driver.find_element_by_id('icp-touch-link-cop')
option.click()

# may need time sleep
time.sleep(0.5)

# select currency setting
currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
currency.click()

# select hkd as currency
hkd = driver.find_element_by_xpath(
    '//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]'
)
hkd.click()

# save change
save = driver.find_element_by_id('icp-btn-save')
save.click()

# may need time sleep
time.sleep(0.5)

# select 'women's fashion' as department
department = driver.find_element_by_xpath(
    '//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]'
)
department.click()

# may need time sleep
time.sleep(0.1)

# activewear
category = 'sweatshirts-hoodies'
    
# load data
df_ck = pd.read_csv(f'data/{category}.csv')     

list_recommend_names = []
list_recommend_prices = []
list_recommend_img_files = []
list_recommend_urls = []

## scraping all links
for i,key_words in enumerate(df_ck['key_words']):

    start_time = time.time()

    # input website
    URL = f'https://www.amazon.com/s?k={key_words}&i=fashion-womens-intl-ship&ref=nb_sb_noss_2'
    driver.get(URL)

    # may need time sleep
    time.sleep(0.5)

    # get links
    links = []
    link_paths = driver.find_elements_by_xpath('//div/h2/a[@class="a-link-normal a-text-normal"]')
    for link in link_paths:
        x = link.get_attribute('href')
        links.append(x)
    
    ## scrape every product details

    names = []
    prices = []
    img_files = []
    index = 0

    for link in links:
        
        # input website
        driver.get(link)

        # product names
        try:
            name = driver.find_elements_by_tag_name('h1')
            name = name[0].text
        except:
            name = None

        # product price
        try:
            price = driver.find_elements_by_xpath('//td[@class="a-span12"]')
            price = price[0].text
        except:
            price = None
        
        # product image
        try:
            # size
            img_width,img_height = 300,300

            # get image
            img = driver.find_elements_by_xpath('//img[@id="landingImage"]')
            img = img[0]

            # 'src' = get image source
            src = img.get_attribute('src')

            # download image
            urllib.request.urlretrieve(src, f'image/Amazon/{index}.png')

            # resize image (smaller size)
            ori_img = Image.open(f'image/Amazon/{index}.png')
            resize_img = ori_img.resize((img_width,img_height))
            resize_img.save(f'image/Amazon/{index}.png')

            img_file = f'image/Amazon/{index}.png'
        except:
            img_file = None

        # append to lists
        names.append(name)
        prices.append(price)
        img_files.append(img_file)

        # update index
        index += 1

    # create dataframe
    df_amazon = pd.DataFrame({
        'name': names,
        'price': prices,
        'img_file': img_files,
        'url': links
    })

    # remove row with missing values
    df_amazon.dropna(inplace=True)
    df_amazon.reset_index(drop=True, inplace=True)

    # products recommender
    recommend_names, recommend_prices, recommend_img_files, recommend_urls = product_recommender(
        i,df_ck.iloc[i],df_amazon,model
    )

    ## output lists for creating dataframe for final deployment use
    list_recommend_names.append(recommend_names)
    list_recommend_prices.append(recommend_prices)
    list_recommend_img_files.append(recommend_img_files)
    list_recommend_urls.append(recommend_urls)
    
    end_time = time.time()
    print(f'({i+1}/{len(df_ck)}) finished, execution time: {round(end_time - start_time,2)}s')

print('Execution Completed')

In [29]:
# create dataframe

names = df_ck['name'].to_list()
prices = df_ck['price'].to_list()
img_files = df_ck['img_file'].to_list()
urls = df_ck['url'].to_list()

df_recommender = pd.DataFrame({
    'name': names,
    'price': prices,
    'img_file': img_files,
    'url': urls,
    'recommend_names': list_recommend_names,
    'recommend_prices': list_recommend_prices,
    'recommend_img_files': list_recommend_img_files,
    'recommend_urls': list_recommend_urls
})

def remove_list(x):
    if '", "' in x:
        return x.strip('["').strip('"]').split('", "')
    if "', '" in x:
        return x.strip("['").strip("']").split("', '")
    if '\', "' in x:
        return x.strip("['").strip('"]').split('\', "')
    if "\", '" in x:
        return x.strip('["').strip("']").split("\", '")

# expanding recommendations into separate columns

recommend_columns = ['recommend_names', 'recommend_prices', 'recommend_img_files', 'recommend_urls']

for column in recommend_columns:
    df_recommender[f'{column[:-1]}1'] = df_recommender[column].apply(lambda x: remove_list(x)[0])
    df_recommender[f'{column[:-1]}2'] = df_recommender[column].apply(
        lambda x: remove_list(x)[1] if len(remove_list(x)) == 2 else None
    )
    df_recommender.drop([column], axis=1, inplace=True)

# save as csv
df_recommender.to_csv(f'data/{category}_recommender.csv', index=False)

In [110]:
# close web browser
# chrome driver seems to be automatically closed after running

# driver.close()

In [ ]:
# call the final function as 'main'

In [ ]:
# END